In [10]:
import pandas as pd 
import os
from pathlib import Path
from dotenv import load_dotenv

#Local
from config import Config
from data_processing import data_processing
from utils import remove_dirs, check_and_create_dirs
from question_generator import question_generator

#Setting
pd.set_option('display.max_colwidth', None)

#CONFIG
load_dotenv()
api_key = os.getenv("PERPLEXITY_API_KEY")

cfg = Config(
    project_name="session_1",
    input_dir="data",
    api_key= api_key,
    n_questions_per_file=1,
    n_page_summary=3,
    chunk_size=5000,
    chunk_overlap=500
)

# remove_dirs(cfg) # This will delete the output directories if they exist
check_and_create_dirs(cfg)


Folder 'session_1' and 'session_1\chunks' has been created.


In [11]:
data_processing(api_key, cfg)

Processing file: 2024-amazon-annual-report-10K.pdf 1/10
Saving chunks to session_1\chunks\2024-amazon-annual-report-10K.json
Saving metadata to session_1\metadata.json
Processing file: 2024-apple-annual-report-10K.pdf 2/10
Saving chunks to session_1\chunks\2024-apple-annual-report-10K.json
Saving metadata to session_1\metadata.json
Processing file: 2024-cisco-full-annual-report.pdf 3/10
Saving chunks to session_1\chunks\2024-cisco-full-annual-report.json
Saving metadata to session_1\metadata.json
Processing file: 2024-google-annual-report-10K.pdf 4/10
Saving chunks to session_1\chunks\2024-google-annual-report-10K.json
Saving metadata to session_1\metadata.json
Processing file: 2024-meta-full-annual-report.pdf 5/10
Saving chunks to session_1\chunks\2024-meta-full-annual-report.json
Saving metadata to session_1\metadata.json
Processing file: 2024-netflix-annual-report-10K.pdf 6/10
Saving chunks to session_1\chunks\2024-netflix-annual-report-10K.json
Saving metadata to session_1\metadata

In [12]:
df = question_generator(cfg)

Generating 1 question for file: 2024-amazon-annual-report-10K.pdf. 1/10
Generating 1 question for file: 2024-apple-annual-report-10K.pdf. 2/10
Generating 1 question for file: 2024-cisco-full-annual-report.pdf. 3/10
Generating 1 question for file: 2024-google-annual-report-10K.pdf. 4/10
Generating 1 question for file: 2024-meta-full-annual-report.pdf. 5/10
Generating 1 question for file: 2024-netflix-annual-report-10K.pdf. 6/10
Generating 1 question for file: 2024-nvidia-annual-report-10K.pdf. 7/10
Generating 1 question for file: 2024-oracle-annual-report-10K.pdf. 8/10
Generating 1 question for file: 2024-reddit-annual-report-10K.pdf. 9/10
Generating 1 question for file: 2024-tsla-annual-report-10K.pdf. 10/10
Questions saved to session_1\questions.csv


In [13]:
df['question'].head()

0                                                                                                                                                                                                             What types of financial statements are included in Amazon's 2024 financial report, as outlined in the document's index to consolidated financial statements?
1                                                                                                                                                                       What forward-looking statements are included in Apple Inc.'s Form 10-K for the fiscal year ended September 28, 2024, and how do they relate to potential future impacts on the company's business?
2    Based on the provided chunk regarding Cisco Systems’ fiscal year 2024 report, here is a question generated according to your requirements:\n\n**How did Cisco Systems’ management and principal officers assess the effectiveness of the company’s disclosure controls and pr

## MODEL 1

In [14]:
from user_models.example_1.model import run_qa_pipeline
from user_models.createplot import plot
from user_models.example_1.qa import create_chain
from user_models.evaluate import evaluate_answer

In [15]:
MODEL_1_CONFIG = {
    "model_name": "model_1",
    "model_type": "faiss",
    
    # Data paths
    "pdf_folder": "./data",
    "vector_store_path": "faiss_index_open",
    "questions_file": "session_1/questions.csv",
    "log_file": "qa_log_1.csv",
    "final_output_file": "final_1.csv",
    
    # Text processing
    "chunk_size": 5000,
    "chunk_overlap": 500,
    "text_splitter_type": "recursive",  # "recursive" or "token"
    "document_loader": "pymupdf",  # "pymupdf" or "pypdf"
    
    # Embeddings
    "embedding_model": "sentence-transformers/all-mpnet-base-v2",
    "embedding_type": "huggingface",
    "embedding_device": "cpu",
    "normalize_embeddings": False,
    
    # LLM settings
    "llm_model": "sonar",
    "pplx_api_key": "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",
    "temperature": 0.2,
    
    # Retrieval settings
    "retriever_k": 3,
    "use_compression": False,
    "use_custom_prompt": False,
    
    # Processing settings
    "recreate_vector_store": False,
    "max_retries": 3
}

# Model 2 Configuration (Chroma + BGE Embeddings + Compression)
MODEL_2_CONFIG = {
    "model_name": "model_2",
    "model_type": "chroma",
    
    # Data paths
    "pdf_folder": "./data",
    "vector_store_path": "chroma_index_finance",
    "questions_file": "session_1/questions.csv",
    "log_file": "qa_log_2.csv",
    "final_output_file": "final_2.csv",
    
    # Text processing
    "chunk_size": 5000,
    "chunk_overlap": 500,
    "text_splitter_type": "token",
    "document_loader": "pypdf",
    
    # Embeddings
    "embedding_model": "BAAI/bge-base-en",
    "embedding_type": "bge",
    "embedding_device": "cpu",
    "normalize_embeddings": True,
    
    # LLM settings
    "llm_model": "sonar",
    "pplx_api_key": "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",
    "temperature": 0.2,
    
    # Retrieval settings
    "retriever_k": 5,
    "use_compression": True,
    "use_custom_prompt": True,
    "custom_prompt": """
    You are a professional financial advisor with expertise in corporate finance, investment analysis, and career development in finance-related roles.

    Use only the information provided in the context to answer the user's question.
    Do not make assumptions or fabricate any details.

    Respond clearly and professionally, as if advising a client on their financial career or investment decisions.

    {context}

    Question: {question}

    If the answer is not explicitly stated in the context, respond with: "I don't know based on the provided document".
    """,
    
    # Processing settings
    "recreate_vector_store": False,
    "max_retries": 3
}

In [16]:
from user_models.qa import create_unified_chain
from user_models.model import run_qa_pipeline
from user_models.evaluate import evaluate_answer
from user_models.createplot import plot

def run_model_experiment(config, experiment_name=""):
    """Run experiment with given configuration"""
    
    print(f"\n{'='*50}")
    print(f"Running {experiment_name or config['model_name']} Experiment")
    print(f"{'='*50}")
    
    # Create QA chain
    qa_chain = create_unified_chain(config)
    
    # Run pipeline
    df, final_df = run_qa_pipeline(
        qa_chain=qa_chain,
        questions_file=config["questions_file"],
        log_file=config["log_file"],
        final_output_file=config["final_output_file"],
        evaluate_answer_func=evaluate_answer,
        max_retries=config["max_retries"]
    )
    
    # Create plots
    try:
        plot(final_df)
        print(f"Plots created successfully for {config['model_name']}")
    except Exception as e:
        print(f"Error creating plots for {config['model_name']}: {e}")
    
    return df, final_df

In [23]:
# Run Model 1 (FAISS + HuggingFace)
df1, final_df1 = run_model_experiment(MODEL_1_CONFIG, "Model 1 (FAISS)")
    
# Run Model 2 (Chroma + BGE + Compression)
df2, final_df2 = run_model_experiment(MODEL_2_CONFIG, "Model 2 (Chroma)")


Running Model 1 (FAISS) Experiment
Loading existing vector store for model_1...
Vector store loaded from: faiss_index_open
Using standard retriever
QA chain created successfully for model_1
=== Starting QA Pipeline ===
Cleared existing log file: qa_log_1.csv
Processing 10 questions from session_1/questions.csv
Processing question 1/10: What types of financial statements are included in...
Processing question 2/10: What forward-looking statements are included in Ap...
Processing question 3/10: Based on the provided chunk regarding Cisco System...
Processing question 4/10: How do evolving and uncertain regulations in areas...
Processing question 5/10: How does Meta Platforms, Inc. address the limitati...
Processing question 6/10: What exhibits related to Netflix, Inc.’s fiscal ye...
Processing question 7/10: How have new US export control restrictions in 202...
Processing question 8/10: What were the key components and figures of Oracle...
Processing question 9/10: What risks does Reddi

Plots created successfully for model_1

Running Model 2 (Chroma) Experiment
Loading existing vector store for model_2...


c:\Users\alice\perplexity_hack\user_models\qa.py:98: LangChainDeprecationWarning:

The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.



Vector store loaded from: chroma_index_finance
Using compression retriever
Using custom prompt template
QA chain created successfully for model_2
=== Starting QA Pipeline ===
Cleared existing log file: qa_log_2.csv
Processing 10 questions from session_1/questions.csv
Processing question 1/10: What types of financial statements are included in...
Processing question 2/10: What forward-looking statements are included in Ap...
Processing question 3/10: Based on the provided chunk regarding Cisco System...
Processing question 4/10: How do evolving and uncertain regulations in areas...
Processing question 5/10: How does Meta Platforms, Inc. address the limitati...
Processing question 6/10: What exhibits related to Netflix, Inc.’s fiscal ye...
Processing question 7/10: How have new US export control restrictions in 202...
Processing question 8/10: What were the key components and figures of Oracle...
Processing question 9/10: What risks does Reddit, Inc. identify in its 2024 ...
Processing q

Plots created successfully for model_2


In [21]:
final_df1.head(10)

question  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   evaluation_comments
0                                                                                                                                                                                                           What types of financial statements are included in Amazon's 2024 financial report, as outlined in the document's index to consolidated financial statements?  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   NaN
1                                                                                                                                                                     What forward-looking statements are included in Apple Inc.'s Form 10-K for the fiscal year ended September 28, 2024, and how do they relate to potential future impacts on the company's business?  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       The answer accurately identifies the sections with forward-looking statements, discusses their content and implications, and aligns fully with the referenced document chunk and relevant SEC filings[1][2][3].
2  Based on the provided chunk regarding Cisco Systems’ fiscal year 2024 report, here is a question generated according to your requirements:\n\n**How did Cisco Systems’ management and principal officers assess the effectiveness of the company’s disclosure controls and procedures for fiscal year 2024, and what was their conclusion regarding those controls?**  ...                                                                                                                                                                                                                                                          The answer accurately reflects the information from the provided document chunk, detailing the management's assessment process, the framework used (COSO 2013), the exclusion of Splunk Inc. in the evaluation per SEC guidance, and the conclusion that Cisco's internal control over financial reporting was effective as of July 27, 2024. It also correctly states PricewaterhouseCoopers LLP's concurring audit opinion. The response fully answers the question clearly

In [22]:
final_df2.head(10)

question  ...                           evaluation_error
0                                                                                                                                                                                                           What types of financial statements are included in Amazon's 2024 financial report, as outlined in the document's index to consolidated financial statements?  ...                                        NaN
1                                                                                                                                                                     What forward-looking statements are included in Apple Inc.'s Form 10-K for the fiscal year ended September 28, 2024, and how do they relate to potential future impacts on the company's business?  ...                                        NaN
2  Based on the provided chunk regarding Cisco Systems’ fiscal year 2024 report, here is a question generated according to your requirements:\n\n**How did Cisco Systems’ management and principal officers assess the effectiveness of the company’s disclosure controls and procedures for fiscal year 2024, and what was their conclusion regarding those controls?**  ...                                        NaN
3                                                                                How do evolving and uncertain regulations in areas such as AI, competition, consumer protection, content moderation, data privacy, news, and sustainability impact Alphabet Inc.'s business operations and risks, particularly as reported for the fiscal year ended December 31, 2024?  ...  Expecting value: line 1 column 1 (char 0)
4                                                                                                                                              How does Meta Platforms, Inc. address the limitations and risks associated with forward-looking statements and the use of non-GAAP financial measures in its financial results for the fourth quarter and full year 2024?  ...                                        NaN
5                                                                                                                                                             What exhibits related to Netflix, Inc.’s fiscal year ended December 31, 2024, are included in the annual report, and what types of certifications and plans are filed with the SEC as part of this report?  ...                                        NaN
6                                                                                                  How have new US export control restrictions in 2024 negatively impacted NVIDIA Corporation's competitive position compared to certain competitors, and what specific risks do these controls pose for its future business and results in the China market and beyond?  ...                                        NaN
7                                                                                                                             What were the key components and figures of Oracle Corporation's consolidated statements of cash flows for the fiscal year ended May 31, 2024, including net income, operating cash flows, investing activities, and financing activities?  ...                                        NaN
8                                                                                                                                                                            What risks does Reddit, Inc. identify in its 2024 fiscal year report related to the dependency on its senior management team and key employees, particularly concerning CEO Steven Huffman?  ...                                        NaN
9                                                                                                                                                                                                                 What was the main reason for the increase in net cas

## MODEL 2

In [20]:
from user_models.example_1.model import qa, finalize
from user_models.createplot import plot
from user_models.example_2.qa import QA2
from user_models.evaluate import evaluate_answer

df2 = qa(QA2)
final_df2 = finalize(evaluate_answer, df2)

ImportError: cannot import name 'QA2' from 'user_models.example_2.qa' (c:\Users\alice\perplexity_hack\user_models\example_2\qa.py)

In [ ]:
final_df2.head(10)

,question,top_k_chunk,answer,evaluation_factual_correctness_score,evaluation_completeness_score,evaluation_clarity_score,evaluation_comments
0,"How could challenges in hiring, retaining, and...",27; 1; 39,"Challenges in hiring, retaining, and managing ...",5,5,5,The answer thoroughly addresses how challenges...
1,What was the total lease liability for Apple I...,154; 386; 157,"As of September 28, 2024, Apple Inc. had fixed...",2,2,5,The answer correctly states the total lease li...
2,What are the key competitive challenges Cisco ...,321; 293; 248,"In 2024, Cisco faced several key competitive c...",4,4,5,"The answer is largely accurate, but lacks spec..."
3,Based on the Alphabet Inc. fiscal year ended D...,528; 527; 497,For Alphabet Inc. in the fiscal year ended Dec...,2,3,4,The answer provides specific per-share figures...
4,What were the key financial results reported b...,544; 552; 549,Here are the key financial results reported by...,4,4,5,"Most facts are correct and sourced, but there ..."
5,What operational and strategic challenges did ...,610; 609; 617,"Based on the available information, Netflix fa...",5,4,5,The answer correctly identifies all major oper...
6,"Question: \r\nWhere, as required by law, does...",768; 660; 723,NVIDIA Corporation files lobbying disclosure r...,5,5,5,The answer correctly states that NVIDIA files ...
7,What highlights or key financial metrics are i...,1022; 886; 1018,Oracle Corporation's fiscal year 2024 Form 10-...,5,5,5,The answer accurately presents key financial h...
8,"How does Reddit, Inc. plan to manage the poten...",1121; 1133; 1120,"Reddit, Inc. acknowledges the risk of user dec...",2,1,3,The model answer presents a comprehensive and ...
9,"Which Tesla, Inc. executives signed the compan...",1337; 1425; 1250,"The following Tesla, Inc. executives and direc...",5,5,5,"The model answer accurately lists all Tesla, I..."


In [ ]:
plot(final_df2)